In [ ]:
!pip install pyautogen

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.3/119.3 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.9/101.9 kB 9.6 MB/s eta 0:00:00


In [ ]:
!pip install autogen-agentchat

In [33]:
!pip install autogen-ext[openai] requests -q

In [34]:
import asyncio
import requests
from typing import Annotated
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import MaxMessageTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_core.models import ModelCapabilities
from autogen_core.tools import FunctionTool

In [35]:
from google.colab import userdata
GROQ_API_KEY = userdata.get("GROQ_API_KEY")
WEATHER_API_KEY = userdata.get("WEATHER_API_KEY")

In [36]:
# ============== Weather Function ==============
def get_weather(city: Annotated[str, "City name"]) -> str:
    """Get weather for a city"""
    try:
        url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&appid={WEATHER_API_KEY}&units=metric"
        response = requests.get(url, timeout=10)

        if response.status_code == 200:
            data = response.json()
            temp = data["main"]["temp"]
            condition = data["weather"][0]["description"]
            return f"Weather in {city}: {temp}°C, {condition}"
        else:
            return f"Error: Could not find weather for {city}"
    except:
        return "Error: Could not fetch weather data"

In [37]:
# ============== Setup Agent ==============
groq_capabilities = ModelCapabilities(
    vision=False,
    function_calling=True,
    json_output=True
)

model_client = OpenAIChatCompletionClient(
    model="llama-3.3-70b-versatile",
    api_key=GROQ_API_KEY,
    base_url="https://api.groq.com/openai/v1",
    model_capabilities=groq_capabilities
)

weather_agent = AssistantAgent(
    name="weather_agent",
    model_client=model_client,
    tools=[FunctionTool(get_weather, description="Get weather for a city")],
    system_message="You are a weather assistant. Use the get_weather tool to get weather data and provide helpful insights."
)

In [40]:
# ============== Run ==============
async def run():
    print("=== Weather Assistant Demo ===\n")

    termination = MaxMessageTermination(max_messages=3)
    team = RoundRobinGroupChat([weather_agent], termination_condition=termination)

    result = await team.run(task="What's the weather in Vadodara?")

    for message in result.messages:
        if message.source == "weather_agent":
            print(f"Assistant: {message.content}")

# Run
await run()

=== Weather Assistant Demo ===

Assistant: [FunctionCall(id='yh8ptacdp', arguments='{"city":"Vadodara"}', name='get_weather')]
Assistant: [FunctionExecutionResult(content='Weather in Vadodara: 23.98°C, mist', name='get_weather', call_id='yh8ptacdp', is_error=False)]
Assistant: Weather in Vadodara: 23.98°C, mist
Assistant: The current weather in Vadodara is 23.98°C with mist.
